### Banco de Preços em Saúde - BPS
O Banco de Preço em Saúde - BPS é um sistema de registro de informações de compras públicas e privadas de medicamentos e dispositivos médicos. Sua principal finalidade é possibilitar o uso de informações de compras públicas e privadas de medicamentos e dispositivos médicos, a fim de subsidiar a compra pública mais eficiente no setor saúde, pelos entes federados e instituições de saúde.

### Dicionário de Dados — Banco de Preços em Saúde (BPS)

A tabela abaixo descreve as colunas presentes no banco de dados BPS:

| **Coluna Original** | **Nome Final (Normalizado)** | **Tipo SQL** | **Descrição** |
|-------------------|------------------------------|--------------|---------------|
| `Código BR` | `codigo_catmat` | `VARCHAR(20)` | Código único do item conforme Catálogo de Materiais (CATMAT). |
| `Descrição CATMAT` | `descricao_catmat` | `VARCHAR(255)` | Nome e características padronizadas do medicamento ou insumo. |
| `Unidade de Fornecimento` | `unidade_fornecimento` | `VARCHAR(50)` | Unidade comercial do item (ex.: comprimido, ampola). |
| `Genérico` | `indicador_generico` | `BOOLEAN` ou `CHAR(1)` | Indica se o item é medicamento genérico (Anvisa). |
| `Código CMED` | `codigo_cmed_anvisa` | `VARCHAR(20)` | Número de registro na Anvisa para comercialização. |
| `Data da Compra` | `data_compra` | `DATE` | Data em que a compra foi realizada pela instituição. |
| `Modalidade da Compra` | `modalidade_compra` | `VARCHAR(50)` | Tipo de processo de compra (licitação, pregão, compra direta, etc.). |
| `Data de Inserção` | `data_insercao_bps` | `DATE` | Data em que a informação foi inserida no sistema BPS. |
| `Tipo Compra` | `tipo_compra` | `VARCHAR(30)` | Classificação da compra (emergencial, regular, etc.). |
| `Fabricante` | `nome_fabricante` | `VARCHAR(150)` | Razão social do fabricante do produto. |
| `CNPJ Fabricante` | `cnpj_fabricante` | `VARCHAR(14)` | CNPJ do fabricante. |
| `Fornecedor` | `nome_fornecedor` | `VARCHAR(150)` | Razão social da empresa que forneceu o item. |
| `CNPJ Fornecedor` | `cnpj_fornecedor` | `VARCHAR(14)` | CNPJ do fornecedor. |
| `Nome Instituição` | `nome_instituicao_compradora` | `VARCHAR(200)` | Nome da instituição que realizou a compra. |
| `CNPJ Instituição` | `cnpj_instituicao` | `VARCHAR(14)` | CNPJ da instituição compradora. |
| `Município Instituição` | `municipio_instituicao` | `VARCHAR(60)` | Município onde a instituição compradora está localizada. |
| `UF` | `uf_instituicao` | `CHAR(2)` | Unidade Federativa da instituição compradora. |
| `Qtd Itens Comprados` | `quantidade_comprada` | `INT` | Quantidade total do item adquirido na transação. |
| `Preço Unitário` | `preco_unitario` | `DECIMAL(10,2)` | Valor pago por unidade do item adquirido. |

In [ ]:
import pandas as pd

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import geopandas as gpd

In [ ]:
bps = pd.read_csv('bps2025.csv', sep=';', encoding='latin1')


Resumo dos dados do ano de 2025

In [ ]:
bps.info()

Visualização de relação de compras por estado, poder observar compras realizadas pelos estados

In [ ]:




url_geojson = "https://raw.githubusercontent.com/codeforamerica/click_that_hood/master/public/data/brazil-states.geojson"
mapa = gpd.read_file(url_geojson)

estado_to_uf = {
    'Acre': 'AC', 'Alagoas': 'AL', 'Amapá': 'AP', 'Amazonas': 'AM', 'Bahia': 'BA',
    'Ceará': 'CE', 'Distrito Federal': 'DF', 'Espírito Santo': 'ES', 'Goiás': 'GO',
    'Maranhão': 'MA', 'Mato Grosso': 'MT', 'Mato Grosso do Sul': 'MS',
    'Minas Gerais': 'MG', 'Pará': 'PA', 'Paraíba': 'PB', 'Paraná': 'PR',
    'Pernambuco': 'PE', 'Piauí': 'PI', 'Rio de Janeiro': 'RJ',
    'Rio Grande do Norte': 'RN', 'Rio Grande do Sul': 'RS', 'Rondônia': 'RO',
    'Roraima': 'RR', 'Santa Catarina': 'SC', 'São Paulo': 'SP', 'Sergipe': 'SE',
    'Tocantins': 'TO'
}
mapa["uf"] = mapa["name"].map(estado_to_uf)


dados = bps.groupby('uf').agg(
    media_itens=('qtd_itens_comprados', lambda x: x.sum() / x.count()),
    preco_medio=('preco_total', lambda x: x.sum() / x.count()),
    preco_medio_unit=('preco_unitario', 'mean')
).reset_index()


ufs = pd.DataFrame({'uf': list(estado_to_uf.values())})
dados = ufs.merge(dados, on='uf', how='left')
dados.fillna(0, inplace=True)


mapa = mapa.merge(dados, on='uf', how='left')



fig, axes = plt.subplots(1, 3, figsize=(22, 7))

metricas = [
    ("media_itens", "Média de Itens por Compra"),
    ("preco_medio", "Preço Médio Total por Compra (R$)"),
    ("preco_medio_unit", "Preço Médio Unitário (R$)")
]

for ax, (col, titulo) in zip(axes, metricas):
    media_nacional = dados[col].mean()

    mapa.plot(
        column=col,
        cmap='bwr',
        linewidth=0.8,
        ax=ax,
        legend=True,
        edgecolor='black',
        missing_kwds={"color": "lightgrey", "edgecolor": "black", "label": "Sem dados"}
    )

    # Força centro no valor médio nacional (B1)
    vmin = dados[col].min()
    vmax = dados[col].max()
    norm_central = max(abs(vmin - media_nacional), abs(vmax - media_nacional))
    ax.set_title(titulo)
    ax.axis('off')

plt.suptitle("Indicadores de Compras por UF ", fontsize=16)
plt.tight_layout()
plt.show()


Observação dos valores médios gastos

In [ ]:
gastos_uf = bps.groupby("uf")["preco_total"].sum().sort_values(ascending=False).head(10)
print(gastos_uf)

gastos_uf.plot(kind="bar", figsize=(8,4), title="Top 10 Estados por Gastos Totais (R$)")
plt.show()


Qual o fornecedor com maior receita media

In [ ]:
forn = bps.groupby("fornecedor")["preco_total"].sum().sort_values(ascending=False).head(10)
print(forn)

forn.plot(kind="bar", figsize=(8,4), title="Top 10 Fornecedores por Receita (R$)")
plt.show()


Observação da modalidade de compra mais comum de medicamentos

In [ ]:
mod = bps.groupby("modalidade_compra").agg(
    total_gasto=("preco_total", "sum"),
    itens=("qtd_itens_comprados", "sum"),
    media_preco_unit=("preco_unitario", "mean")
).sort_values("total_gasto", ascending=False)

print(mod)

mod["total_gasto"].plot(kind="bar", figsize=(8,4), title="Total Gasto por Modalidade de Compra")
plt.show()


Quais os produtos mais comprados e de maior interesse no ano

In [ ]:
top_qtd = bps.groupby("descricao_catmat")["qtd_itens_comprados"].sum().sort_values(ascending=False).head(10)
top_price = bps.groupby("descricao_catmat")["preco_total"].sum().sort_values(ascending=False).head(10)

print("Mais comprados:")
print(top_qtd)

print("\nMaior gasto:")
print(top_price)

top_qtd.plot(kind="bar", figsize=(8,4), title="Top 10 Produtos Mais Comprados")
plt.show()

top_price.plot(kind="bar", figsize=(8,4), title="Top 10 Produtos com Maior Gasto Total")
plt.show()


Descobrir o medicamento mais comprado para cada estado

In [ ]:


bps_meds = (
    bps.groupby(["uf", "descricao_catmat"], as_index=False)
      .agg(total_comprado=("qtd_itens_comprados", "sum"))
)

# Para cada estado, pega o medicamento mais comprado
bps_top_meds_estado = (
    bps_meds.sort_values(["uf", "total_comprado"], ascending=[True, False])
           .groupby("uf")
           .first()
           .reset_index()
)

print("Medicamento mais comprado por estado:")
print(bps_top_meds_estado)


In [ ]:
missing_counts = bps.isna().sum().sort_values(ascending=False)

# Número total de linhas (seguro)
total = len(bps)

print("Resultado: \n")
for col, count in missing_counts.items():
        pct = (count / total) * 100
        print(f"{col}: {count} ({pct:.2f}%) valores ausentes")

print(f"\nTotal de linhas analisadas: {total:,}")


In [ ]:
#Analise de variaveis
cat_cols = bps.select_dtypes(include='object').columns

for col in cat_cols:
    print(f"\n--- {col} ---")
    print(bps[col].value_counts(normalize=True).head(1000))

In [ ]:
from sklearn.preprocessing import MinMaxScaler
import numpy as np
import matplotlib.pyplot as plt

# Calcula percentual de nulos por coluna
scaler = MinMaxScaler()
missing_percent = (missing_counts / total) * 100
missing_percent = missing_percent.sort_values(ascending=False)

# ✅ Remove colunas com 0%
missing_percent = missing_percent[missing_percent > 0]

# Cria figura
plt.figure(figsize=(5, 4))
plt.bar(missing_percent.index, missing_percent.values)
plt.xticks(rotation=90)
plt.ylabel('% de valores ausentes')
plt.title('Percentual de valores ausentes por coluna')
plt.tight_layout()
plt.show()

In [ ]:
bps[['preco_unitario', 'preco_total', 'qtd_itens_comprados']].describe()


In [ ]:
bps['descricao_catmat'].value_counts().head(10)


In [ ]:
bps.nunique()




In [ ]:
bps['generico'].value_counts(dropna=False)


In [ ]:
bps.groupby('nome_instituicao')['preco_total'].mean().sort_values(ascending=True).head(10)


In [ ]:
modal_counts = bps['modalidade_compra'].value_counts()
print(modal_counts.head(10))

plt.figure(figsize=(8, 4))
modal_counts.plot(kind='bar')
plt.ylabel('Frequência')
plt.title('Modalidade de compra mais frequente')
plt.tight_layout()
plt.show()


In [ ]:
top_fornecedores = bps.groupby('fornecedor')['qtd_itens_comprados'] \
                     .sum() \
                     .sort_values(ascending=True) \
                     .head(10)

print(top_fornecedores)

plt.figure(figsize=(10, 4))
top_fornecedores.plot(kind='bar')
plt.ylabel('Total de itens comprados')
plt.title('Top 10 fornecedores por quantidade de itens')
plt.tight_layout()
plt.show()


In [ ]:
cat_counts = bps['descricao_catmat'].value_counts().head(10)
print(cat_counts)

plt.figure(figsize=(10, 4))
cat_counts.plot(kind='bar')
plt.ylabel('Frequência')
plt.title('Top 10 categorias mais compradas')
plt.tight_layout()
plt.show()


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Seleciona apenas colunas numéricas
bps_num = bps.select_dtypes(include=['float64', 'int64'])

plt.figure(figsize=(10, 6))
sns.heatmap(bps_num.corr(), annot=True, fmt=".2f", cmap="viridis")
plt.title("Mapa de calor das correlações entre variáveis numéricas")
plt.tight_layout()
plt.show()


In [ ]:
preco_medio_inst = bps.groupby('nome_instituicao')['preco_total'].mean().sort_values(ascending=False)

print(preco_medio_inst.head(10))  # Mostra top 10
print("\nQuantidade de instituições:", preco_medio_inst.shape[0])

# Gráfico das 10 com maior preço médio
plt.figure(figsize=(10, 4))
preco_medio_inst.head(10).plot(kind='bar')
plt.ylabel('Preço médio')
plt.title('Top 10 instituições com maior preço médio total')
plt.tight_layout()
plt.show()


In [ ]:
itens_por_uf = bps.groupby('uf')['qtd_itens_comprados'] \
                 .sum() \
                 .sort_values(ascending=False)

print(itens_por_uf)

plt.figure(figsize=(8, 4))
itens_por_uf.plot(kind='bar')
plt.ylabel('Quantidade total de itens')
plt.title('Itens comprados por UF')
plt.tight_layout()
plt.show()


In [ ]:
gasto_por_uf = bps.groupby('uf')['preco_total'] \
                 .sum() \
                 .sort_values(ascending=False)

print(gasto_por_uf)

plt.figure(figsize=(8, 4))
gasto_por_uf.plot(kind='bar')
plt.ylabel('Valor total gasto')
plt.title('Total gasto por UF')
plt.tight_layout()
plt.show()
